In [19]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#data collection
dataset=pd.read_csv("Social_Network_Ads.csv")
print(dataset)

#preprocess
#drop/remove user ID, model creation
dataset=dataset.drop("User ID",axis=1)
dataset

#preprocess Gender to 0 or 1
dataset=pd.get_dummies(dataset,drop_first=True).astype(int)
print(dataset.columns)

#Balanced Dataset or Imbalanced Dataset
print(dataset["Purchased"].value_counts())

#input output split
independent=dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
dependent=dataset[['Purchased']]

print(independent.shape)
print(dependent.shape)

#Train Test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent, dependent, test_size=0.30,random_state=0)

#preprocess standard Scaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

      User ID  Gender  Age  EstimatedSalary  Purchased
0    15624510    Male   19            19000          0
1    15810944    Male   35            20000          0
2    15668575  Female   26            43000          0
3    15603246  Female   27            57000          0
4    15804002    Male   19            76000          0
..        ...     ...  ...              ...        ...
395  15691863  Female   46            41000          1
396  15706071    Male   51            23000          1
397  15654296  Female   50            20000          1
398  15755018    Male   36            33000          0
399  15594041  Female   49            36000          1

[400 rows x 5 columns]
Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')
Purchased
0    257
1    143
Name: count, dtype: int64
(400, 3)
(400, 1)


In [20]:
#Model creation
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
from sklearn.tree import DecisionTreeClassifier

#Model creation using Grid
from sklearn.model_selection import GridSearchCV
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],  
    'max_features': [None, 'sqrt', 'log2'],        
    'splitter': ['best', 'random']
}
classifier = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
# fitting the model for grid search 
classifier.fit(X_train, y_train) 

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': [None, 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [21]:
#Evaluation Matrix: Confusion_matrix
re=classifier.cv_results_
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
#Evaluation Matrix: calculate P R F
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, y_pred)
print(clf_report)

[[71  8]
 [ 4 37]]
              precision    recall  f1-score   support

           0       0.95      0.90      0.92        79
           1       0.82      0.90      0.86        41

    accuracy                           0.90       120
   macro avg       0.88      0.90      0.89       120
weighted avg       0.90      0.90      0.90       120



In [22]:
# print best parameter after tuning 
#print(classifier.best_params_) 
re=classifier.cv_results_
from sklearn.metrics import f1_score
f1_weighted=f1_score(y_test,y_pred,average='weighted')
print("The weighted F1-score for best parameter {}:".format(classifier.best_params_),f1_weighted)

The weighted F1-score for best parameter {'criterion': 'entropy', 'max_features': None, 'splitter': 'best'}: 0.9010268800966474


In [23]:
#Roc = ROC (Receiver Operating Characteristic) Curve
#AUC (Area Under the Curve)
from sklearn.metrics import roc_auc_score
print("ROC-AUC Score:",roc_auc_score(y_test,classifier.predict_proba(X_test)[:,1]))
table=pd.DataFrame.from_dict(re)
table

ROC-AUC Score: 0.9005866008027168


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002596,0.000269,0.010920,0.001571,gini,None,best,"{'criterion': 'gini', 'max_features': None, 's...",0.799537,0.840114,0.804584,0.876368,0.909115,0.845943,0.041987,6
1,0.003466,0.000678,0.010674,0.000519,gini,None,random,"{'criterion': 'gini', 'max_features': None, 's...",0.795918,0.804584,0.730543,0.853485,0.891667,0.815239,0.054707,16
2,0.002829,0.000351,0.010237,0.001525,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.802399,0.766471,0.804584,0.910254,0.853485,0.827438,0.049806,10
3,0.003084,0.000739,0.012633,0.003745,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.857143,0.766471,0.787755,0.821429,0.872761,0.821112,0.040160,12
4,0.002946,0.000083,0.013928,0.003953,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.816207,0.802399,0.787755,0.910254,0.853485,0.834020,0.043928,8
5,0.003906,0.000839,0.011782,0.001026,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.840114,0.802399,0.787755,0.819444,0.841045,0.818151,0.020882,14
6,0.003941,0.000266,0.010350,0.001202,entropy,None,best,"{'criterion': 'entropy', 'max_features': None,...",0.819142,0.840114,0.789949,0.892857,0.927778,0.853968,0.049927,1
7,0.004442,0.001377,0.011253,0.001683,entropy,None,random,"{'criterion': 'entropy', 'max_features': None,...",0.857143,0.787755,0.787755,0.766661,0.928571,0.825577,0.059894,11
8,0.003650,0.000424,0.011406,0.000857,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.855314,0.802399,0.823129,0.821429,0.870721,0.834598,0.024802,7
9,0.003616,0.000719,0.011691,0.001170,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.893878,0.875644,0.806153,0.792857,0.870721,0.847851,0.040440,4


In [12]:
#save model
import pickle
filename="final_Sav_Model.sav"
pickle.dump(classifier,open(filename,'wb'))
load_model=pickle.load(open("final_Sav_Model.sav",'rb'))

#apply StandardScaler to user input
preinput=sc.transform([[46,41000,1]])
print(preinput)
result=load_model.predict(preinput)
result

[[ 0.79762394 -0.8330751   0.99288247]]


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([1])